In [2]:
import flickrapi
import json
import folium
import configparser
from geopy.geocoders import Nominatim
from folium.plugins import HeatMap

In [3]:
config = configparser.ConfigParser()
config.read('config.ini')

api_key = config['API']['API_KEY']
api_secret = config['API']['API_SECRET']

latitude = config['LOC']['LATITUDE']
longitude = config['LOC']['LONGITUDE']

# Initialize the geocoder
geolocator = Nominatim(user_agent="my_geocoder")

location = geolocator.geocode(f"{latitude} {longitude}")
latitude_dd = location.latitude
longitude_dd = location.longitude

print(f"Latitude in Decimal Degrees: {latitude_dd}")
print(f"Longitude in Decimal Degrees: {longitude_dd}")

Latitude in Decimal Degrees: 41.8932408
Longitude in Decimal Degrees: 12.482981487083691


In [5]:
flickr = flickrapi.FlickrAPI(api_key, api_secret, format='json')
flickr.authenticate_via_browser(perms='read')   

In [6]:
photos = []
for page in range(1,11):
    shots = flickr.photos.search(has_geo="1", extras="geo, owner_name", privacy_filter="1", per_page="250",
                                radius_units="km", radius=5, lat=latitude_dd, lon=longitude_dd, accuracy = 16)
    parsed = json.loads(shots.decode('utf-8'))
    photos += parsed['photos']['photo']

In [7]:
styles = ["Stamen Terrain", "Stamen Toner", "Mapbox Bright"]

map_rome = folium.Map(location=[latitude_dd, longitude_dd], # latitude, longitude
                      # tiles="Stamen Terrain",
                      zoom_start = 14) # The bigger the zoom number, the closer in you get

locations = []
for photo in photos:
    # folium.Marker([photo['latitude'], photo['longitude']], 
    #                popup=photo['title']).add_to(map_rome)
    locations.append([photo['latitude'], photo['longitude']])

HeatMap(locations).add_to(map_rome)
map_rome

In [32]:
# Open the text file for reading
with open('./output/loc.txt', 'r') as file:
    # Initialize an empty list to store the coordinates
    coordinates = []

    # Read each line in the file
    for line in file:
        # Split the line into longitude and latitude using a comma as the delimiter
        parts = line.strip().split(',')
        
        # Convert the parts to floating-point numbers
        longitude = float(parts[0])
        latitude = float(parts[1])
        
        # Append the longitude and latitude as a tuple to the list
        coordinates.append((longitude, latitude))

styles = ["Stamen Terrain", "Stamen Toner", "Mapbox Bright"]

map_rome = folium.Map(location=[latitude_dd, longitude_dd], # latitude, longitude
                      # tiles="Stamen Terrain",
                      zoom_start = 14) # The bigger the zoom number, the closer in you get

for loc in coordinates:
    folium.Circle(location=[loc[1], loc[0]], radius=500, color='blue', fill=True).add_to(map_rome)
    
map_rome